In [4]:
import os

#if running on colab
colab = 'google.colab' in str(get_ipython())

if colab:
    content_path = "/content/drive/MyDrive/Core Images"
    project_path = "/content/"
else:
    project_path = os.getcwd()
    content_path = project_path + "/data/Core Images"
    
if colab:
    !rm "well-core-image-classification/" -r
    !git clone https://github.com/adhardy/well-core-image-classification.git

In [5]:
import importlib

if colab:
  %cd $(project_path + "well-core-image-classification/")

import well_core_preprocessing as core_images
importlib.reload(core_images)

if colab:
  %cd -

In [6]:
%cd $content_path

#clear directories
!rm slices/* cores/*

#get list of all images in directories
image_dirs = ["42-29- 6/", "3-03- 11/",]
#,"data/Core Images/21-20a- 1/",","data/Core Images/206-01- 2/"] 

#list all the images in each directory
images = []
for dir in image_dirs:
    files = os.listdir(dir)
    dir_images = [os.path.join(dir, f) for f in files]
    images += dir_images

images = sorted(images)

core_x = 730
core_x_size = 6240
core_x_cm = 955
core_y_size = 865
core_x = (core_x,core_x + core_x_size)
core_y= (
    (1470,1470 + core_y_size),
    (2550,2550 + core_y_size),
    (3615,3615 + core_y_size)
)

slice_size = 224
step_size = slice_size/4

cores = core_images.CoreImages(images, core_x, core_y, core_x_cm, slice_size, step_size)

labels = "core_labels.csv"

cores.slice_cores("cores/", "slices/", labels, verbose = 1, save_images=True)

%cd -

/home/adam/well-core-image-classification/data/Core Images
Processing core photo: 3-03- 11/S00066841.jpg
Processing core photo: 3-03- 11/S00066842.jpg
Processing core photo: 3-03- 11/S00066843.jpg
Processing core photo: 3-03- 11/S00066844.jpg
Processing core photo: 3-03- 11/S00066845.jpg
Processing core photo: 3-03- 11/S00066846.jpg
Processing core photo: 3-03- 11/S00066847.jpg
Processing core photo: 3-03- 11/S00066848.jpg
Processing core photo: 42-29- 6/S00085344.jpg
Processing core photo: 42-29- 6/S00085345.jpg
Processing core photo: 42-29- 6/S00085346.jpg
Processing core photo: 42-29- 6/S00085347.jpg
Processing core photo: 42-29- 6/S00085348.jpg
Processing core photo: 42-29- 6/S00085349.jpg
Processing core photo: 42-29- 6/S00085350.jpg
Processing core photo: 42-29- 6/S00085351.jpg
Processing core photo: 42-29- 6/S00085352.jpg
Processing core photo: 42-29- 6/S00085353.jpg
/home/adam/well-core-image-classification
